# Draft for a New DiffEqOperator Interface

The basic idea is instead of implementing operators as `LinearMap` (which is incorrect for affine and quasilinear operators anyways), we mimic the internals of LinearMaps.jl and construct a similar type hierarchy `DiffEqOperator`, which is like an expanded version of `LinearMap`.

The most general form of a `DiffEqOperator` should look like

$$ L(u,p,t)= A_{11}(u,p,t)A_{12}(u,p,t)\ldots A_{1n_1}(u,p,t)u + \ldots + A_{m1}(u,p,t)A_{m2}(u,p,t)\ldots A_{mn_m}(u,p,t)u + b(u,p,t) $$

where each $A_{ij}$ are themselves built up the same way recursively, with the fundamental building blocks being the identity, array and derivative operators. When the affine term $b(u,p,t) = 0$, $L$ is a quasilinear operator as it acts like a linear operator but in fact depends on $u$ (a specific example would be the nonlinear Hamiltonian from density functional theory).

Apart from the interfaces for a (quasi)linear operator (operator arithmetic, `eltype`, `size`, `A_mul_B!`, ...), functionalities specific to `JuliaDiffEq` include:

- Efficient representations (`as_array`) for converting the stencil operators to arrays of the most suitable form.

    - Both `full` and `sparse` enforce one either the full or sparse form, so we need another method that returns the most "appropriate" form.

- Time-dependent operators via the `update_coefficients!` interface.

    - Basic types include an `update_func` field to handle their own updates;
    
    - Composite types (compositions and combinations) call `update_coefficients!` recursively.

- `L(u,p,t)` and `L(du,u,p,t)` functor signature (for use with generic or Krylov ODE solvers).

    - Should always call `update_coefficients!` first;

    - The methods are defined using the "unsafe" `L*u` and `A_mul_B!(du,L,u)` methods defined separately for each type.

## Notes/TODO

- The current implementations are scattered in both DiffEqBase and DiffEqOperators. Should probably move everything inside DiffEqOperators.

- Need to be careful with scalar equations (e.g. in-place style would not be valid).

- Need to consider how `StaticArrays` fit.

- Probably need some convenient functors to use as update functions.

- Iterator interface for `DiffEqCompositeOperator` and `DiffEqOperatorCombination`.

In [1]:
import Base: +, -, *, /, A_mul_B!, eltype, size

abstract type DiffEqOperator{T} end
eltype(::DiffEqOperator{T}) where {T} = T
size(L::DiffEqOperator, k::Int) = k <= 2 ? size(L)[k] : 1 # size(L) is defined separately for each subtype

# The default update function is defined as a functor for easier type dispatching
# For example, DiffEqConstOpCombination below
struct DefaultUpdateFunc end
(::DefaultUpdateFunc)(A,u,p,t) = nothing

# Basic operators

## Array operators (`WrappedMap` in LinearMaps.jl)

Instead of allowing both arrays and scalars, `DiffEqArrayOperator` should only accept arrays for easier handling (one reason among others being `A_mul_B!`, which isn't defined for scalars). The internal scalar coefficient is also dropped in favor of `DiffEqOperatorCombination`. Other than that the interface is basically the same as the current implementation.

In [2]:
struct DiffEqArrayOperator{T,Arr,F} <: DiffEqOperator{T}
    A::Arr
    update_func::F
end
DiffEqArrayOperator(A::AbstractMatrix, update_func=DefaultUpdateFunc()) = 
    DiffEqArrayOperator{eltype(A),typeof(A),typeof(update_func)}(A, update_func)
update_coefficients!(L::DiffEqArrayOperator,u,p,t) = L.update_func(L.A,u,p,t)
size(L::DiffEqArrayOperator) = size(L.A)

# Application
*(L::DiffEqArrayOperator, u::AbstractVector) = L.A * u
A_mul_B!(y::AbstractVector, L::DiffEqArrayOperator, x::AbstractVector) = A_mul_B!(y, L.A, x)

# Representation
as_array(L::DiffEqArrayOperator) = L.A;

## Identity operators (`IdentityMap` in LinearMaps.jl)

They are mainly for right hand side terms of the from $a(t)u$, which we interpret as $(a(t)I)u$. The scalar coefficient part is handled by `DiffEqOperatorCombination`.

In accordance with the `size` interface of other operators, we need to explicitly provide the dimension in the constructor.

In [3]:
struct DiffEqIdOperator{T} <: DiffEqOperator{T}
    M::Int
end
update_coefficients!(L::DiffEqIdOperator,u,p,t) = nothing
size(L::DiffEqIdOperator) = (L.M, L.M)

# Application
*(L::DiffEqIdOperator, u) = u
A_mul_B!(y, L::DiffEqIdOperator, x) = copy!(y, x)

# Representation
as_array(L::DiffEqIdOperator{T}) where {T} = Diagonal(ones(T, L.M, L.M));

## Derivative Operators

The definition of derivative operators should remain mostly unchanged except for `as_array`.

# Operators built upon the basic operators

## Compositions (`CompositeMap` in LinearMaps.jl)

The `DiffEqCompositeOperator` type is basically a tuple of `DiffEqOperator`. The operators are stored in the order of application, so `L1*L2*L3` is stored as `(L3,L2,L1)`.

What's different from `CompositeMap` is that a series of intermediate caches is allocated a priori to avoid the need to allocate memory during `A_mul_B!`.

TODO: 

- Check dimension mismatch errors.

- Allow for custom array types for the caches (adding a `u_prototype` argument to the constructor?).

In [4]:
struct DiffEqCompositeOperator{T} <: DiffEqOperator{T}
    ops::Tuple{Vararg{DiffEqOperator{T}}}
    caches::Vector{Vector{T}}
end
function DiffEqCompositeOperator(ops::DiffEqOperator{T}...) where {T}
    # Initialize the intermediate caches
    caches = Vector{Vector{T}}()
    for op in ops[1:end-1]
        c = Vector{T}(size(op, 1))
        push!(caches, c)
    end
    DiffEqCompositeOperator{T}(ops, caches)
end
update_coefficients!(L::DiffEqCompositeOperator,u,p,t) = foreach(op -> update_coefficients!(op,u,p,t), L.ops)
size(L::DiffEqCompositeOperator) = (size(L.ops[end], 1), size(L.ops[1], 2))

# Application
*(L::DiffEqCompositeOperator, u) = foldl((x, op) -> op*x, u, L.ops) # should we define this using A_mul_B! ?
function A_mul_B!(y, L::DiffEqCompositeOperator, x)
    N = length(L.ops)
    if N == 1
        A_mul_B!(y, L.ops[1], x)
    else
        A_mul_B!(L.caches[1], L.ops[1], x)
        for i in 2:N-1
            A_mul_B!(L.caches[i], L.ops[i], L.caches[i-1])
        end
        A_mul_B!(y, L.ops[end], L.caches[end])
    end
end

# Representation
as_array(L::DiffEqCompositeOperator) = prod(as_array, reverse(L.ops));

## Linear Combinations (`LinearCombination` in LinearMaps.jl)

Like `LinearCombination` (and unlike the `AffineDiffEqOperator` in DiffEqBase.jl), coefficients are explicitly included. The rationale is that operations involving scalars can usually be optimized. The alternative method would be to implement a `DiffEqScalarOperator` type and treat `c_1*A + c_2*B` as a combination of two composite operators `c_1*A` and `c_2*B`.

The coefficients are implemented as vectors instead of tuples so that they can be mutated by the update function (will this cause a performance issue?)

As with the case of composite operators, a cache is used to speed up `A_mul_B!`.

In [5]:
struct DiffEqOperatorCombination{T,F} <: DiffEqOperator{T}
    ops::Tuple{Vararg{DiffEqOperator{T}}}
    coeffs::Vector{T}
    cache::Vector{T}
    update_func::F
end
function DiffEqOperatorCombination(ops::Tuple{Vararg{DiffEqOperator{T}}}, coeffs=nothing, 
        update_func=DefaultUpdateFunc()) where {T}
    if coeffs == nothing
        coeffs = ones(T, length(ops))
    else
        coeffs = collect(coeffs) # make sure is array
    end
    cache = Vector{T}(size(ops[1], 1))
    DiffEqOperatorCombination{T,typeof(update_func)}(ops, coeffs, cache, update_func)
end
function update_coefficients!(L::DiffEqOperatorCombination,u,p,t)
    L.update_func(L.coeffs,u,p,t)
    for op in L.ops
        update_coefficients!(op,u,p,t)
    end
end
size(L::DiffEqOperatorCombination) = size(L.ops[1])

# Application
*(L::DiffEqOperatorCombination, u) = sum(c * (op * u) for (c,op) in zip(L.coeffs,L.ops))
function A_mul_B!(y, L::DiffEqOperatorCombination{T}, x) where {T}
    A_mul_B!(y, L.ops[1], x)
    L.coeffs[1] == one(T) || scale!(L.coeffs[1], y) # is this necessary?
    for (c,op) in zip(L.coeffs[2:end], L.ops[2:end])
        A_mul_B!(L.cache, op, x)
        if c == one(T)
            y .+= L.cache
        else
            @. y += c * L.cache
            # Base.axpy!(c, L.cache, y) # better?
        end
    end
    return y
end

# Representation
as_array(L::DiffEqOperatorCombination) = sum(i -> L.coeffs[i] * as_array(L.ops[i]), 1:length(L.ops));

## Affine operator

In [6]:
struct DiffEqAffineOperator{T,F} <: DiffEqOperator{T}
    A::DiffEqOperator{T}
    b::AbstractVector{T}
    update_func::F
end
DiffEqAffineOperator(A::DiffEqOperator{T}, b::AbstractVector{T}, update_func=DefaultUpdateFunc()) where {T} = 
    DiffEqAffineOperator{T,typeof(update_func)}(A,b,update_func)
update_coefficients!(L::DiffEqAffineOperator,u,p,t) = (update_coefficients!(A,u,p,t); L.update_func(b,u,p,t))
size(L::DiffEqAffineOperator) = size(L.A)

# Application
*(L::DiffEqAffineOperator, u) = L.A * u + b
A_mul_B!(y, L::DiffEqAffineOperator, x) = (A_mul_B!(y,L.A,x); y .+= b)

# Representation
# Ax + b == [A b] * [x;1]
as_array(L::DiffEqAffineOperator) = hcat(as_array(L.A), b);

## Operator arithmetics

Arithmetics on linear combinations with non-constant coefficients are a bit tricky (though not impossible). The constant version is easy and we can dispatch types using the `UnionAll` type definition feature.

In [7]:
const DiffEqConstOpCombination = DiffEqOperatorCombination{T,DefaultUpdateFunc} where {T}

DiffEqOperatorCombination{T,DefaultUpdateFunc} where T

In [8]:
# op + op
+(L1::DiffEqConstOpCombination{T}, L2::DiffEqConstOpCombination{T}) where {T} = DiffEqOperatorCombination(
    tuple(L1.ops..., L2.ops...), vcat(L1.coeffs, L2.coeffs))
+(L1::DiffEqConstOpCombination{T}, L2::DiffEqOperator{T}) where {T} = DiffEqOperatorCombination(
    tuple(L1.ops..., L2), vcat(L1.coeffs, one(T)))
+(L1::DiffEqOperator{T}, L2::DiffEqConstOpCombination{T}) where {T} = DiffEqOperatorCombination(
    tuple(L1, L2.ops...), vcat(one(T), L2.coeffs))
+(L1::DiffEqOperator{T}, L2::DiffEqOperator{T}) where {T} = DiffEqOperatorCombination((L1,L2), ones(T,2))

# scalar * op and op * scalar
*(a::T, L::DiffEqConstOpCombination{T}) where {T} = DiffEqOperatorCombination(L.ops, a*L.coeffs)
*(L::DiffEqConstOpCombination{T}, a::T) where {T} = DiffEqOperatorCombination(L.ops, a*L.coeffs)
*(a::T, L::DiffEqOperator{T}) where {T} = DiffEqOperatorCombination((L,), (a,))
*(L::DiffEqOperator{T}, a::T) where {T} = DiffEqOperatorCombination((L,), (a,))

# op * op
# Note the application order
*(L1::DiffEqCompositeOperator{T}, L2::DiffEqCompositeOperator{T}) where {T} = DiffEqCompositeOperator(L2.ops..., L1.ops...)
*(L1::DiffEqOperator{T}, L2::DiffEqCompositeOperator{T}) where {T} = DiffEqCompositeOperator(L2.ops..., L1)
*(L1::DiffEqCompositeOperator{T}, L2::DiffEqOperator{T}) where {T} = DiffEqCompositeOperator(L2, L1.ops...)
*(L1::DiffEqOperator{T}, L2::DiffEqOperator{T}) where {T} = DiffEqCompositeOperator(L2, L1);

## Operator concatenations

In some applications the domain/grid $\mathbf{u}$ is composed of different regions (e.g. each region corresponding to a different physical region or the same region but with different discrete states)

$$ \mathbf{u} = \begin{pmatrix} \mathbf{u}_1 \\ \mathbf{u}_2 \\ \vdots \\ \mathbf{u}_m \end{pmatrix} $$

For each region $k$, there's a correpsonding differential operator $L_k$, and the combined operator $L$ can be expressed as a block diagonal matrix

$$ L\mathbf{u} = \begin{pmatrix} L_1\mathbf{u}_1 \\ L_2\mathbf{u}_2 \\ \vdots \\ L_m\mathbf{u}_m \end{pmatrix}, \quad L = \begin{pmatrix} L_1 & 0 & \cdots & 0 \\ 0 & L_2 & \cdots & 0 \\ \vdots & \vdots & \ddots & \vdots \\ 0 & 0 & \cdots & L_m \end{pmatrix} $$

Formally, $\mathbf{u}$ is the direct sum $\mathbf{u}_1 \oplus \mathbf{u}_2 \oplus \cdots \oplus \mathbf{u}_m$ and $L$ is the direct sum $L_1 \oplus L_2 \oplus \cdots \oplus L_m$.

Implementation-wise, subarray views (the `@views` macro) are used for faster indexing.

Note: the current version works only for vectors with linear indexing.

In [9]:
import Base.blkdiag
struct DiffEqBlkDiagOperator{T} <: DiffEqOperator{T}
    ops::Tuple{Vararg{DiffEqOperator{T}}}
end
blkdiag(ops::DiffEqOperator{T}...) where {T} = DiffEqBlkDiagOperator{T}(ops)
update_coefficients!(L::DiffEqBlkDiagOperator,u,p,t) = foreach(op -> update_coefficients!(op,u,p,t), L.ops)
size(L::DiffEqBlkDiagOperator) = (sum(op -> size(op,1), L.ops), sum(op -> size(op,2), L.ops))

# Application
function *(L::DiffEqBlkDiagOperator, u::AbstractVector)
    vs = Vector{AbstractVector}()
    istart = 1
    for op in L.ops
        iend = istart + size(op, 2)
        @views push!(vs, op * u[istart:iend-1])
        istart = iend
    end
    reduce(vcat, vs)
end
function A_mul_B!(y::AbstractVector, L::DiffEqBlkDiagOperator, x::AbstractVector)
    istart = 1 # istart & iend tracks index for x
    jstart = 1 # jstart & jend tracks index for y
    for op in L.ops
        iend = istart + size(op, 2); jend = jstart + size(op, 1)
        @views A_mul_B!(y[jstart:jend-1], op, x[istart:iend-1])
        istart = iend; jstart = jend
    end
    return y
end

# Representation
# We need to differentiate between full and sparse case as `Base.blkdiag` only 
# supports sparse matrices
function as_array(L::DiffEqBlkDiagOperator{T}) where {T}
    As = map(as_array, L.ops)
    if all(issparse, As)
        return blkdiag(As...)
    else
        out = zeros(T, size(L))
        istart = 1
        jstart = 1
        for A in As
            iend = istart + size(A, 2); jend = jstart + size(A, 1)
            out[jstart:jend-1, istart:iend-1] = A
            istart = iend; jstart = jend
        end
        return out
    end
end

as_array (generic function with 6 methods)

TODO: In general, we may encounter operators constructed by row/column stacking operators. (possibly interface with BlockArrays.jl?)

## Operators on high-dimensional spaces

When a PDE is discretized on a high-dimensional space, it is natural to think of the solution vector $\mathbf{u}$ as a $D$-dimensional array. A (quasi)linear operator on $\mathbf{u}$ can usually be expressed as the tensor product $L = L_D \otimes L_{D-1} \otimes \cdots \otimes L_1$, where each of the $L_j$ acts on the $j$th dimension of $\mathbf{u}$.

Sometimes it is preferrable to use the "flattened" version of $\mathbf{u}$, `vec(u)` for linear solvers, etc that require vector inputs. The matrix of $L$ in this case can be calculated using the Kronecker product `kron`. It is also possible to create lazy versions of the Kronecker product for the differential operators, which is similar to what `DiffEqCompositeOperator` does. A `DiffEqKronProd` for two operators is implemented below:

In [10]:
import Base.kron
struct DiffEqKronProd{T} <: DiffEqOperator{T}
    A::DiffEqOperator{T}
    B::DiffEqOperator{T}
    cache::Matrix{T}
end
function DiffEqKronProd(A::DiffEqOperator{T}, B::DiffEqOperator{T}) where {T}
    cache = Matrix{T}(size(B,1), size(A,2))
    DiffEqKronProd{T}(A, B, cache)
end
kron(L1::DiffEqOperator{T}, L2::DiffEqOperator{T}) where {T} = DiffEqKronProd(L1, L2)
update_coefficients!(L::DiffEqKronProd,u,p,t) = (update_coefficients!(L.A,u,p,t); update_coefficients!(L.B,u,p,t))
size(L::DiffEqKronProd) = (size(L.A,1) * size(L.B,1), size(L.A,2) * size(L.B,2))

# Application
function *(L::DiffEqKronProd, u::AbstractVector)
    U = reshape(u, size(L.B,2), size(L.A,2))
    V = zeros(eltype(L), size(L.B,1), size(L.A,1))
    # Apply B to each column
    for j = 1:size(U,2)
        L.cache[:,j] = L.B * U[:,j]
    end
    # Apply A to each row
    for i = 1:size(V,1)
        V[i,:] = L.A * L.cache[i,:]
    end
    return vec(V)
end
function A_mul_B!(y::AbstractVector, L::DiffEqKronProd, x::AbstractVector)
    X = reshape(x, size(L.B,2), size(L.A,2))
    Y = reshape(y, size(L.B,1), size(L.A,1)) # reshape creates an alias
    # Apply B to each column
    for j = 1:size(X,2)
        @views A_mul_B!(L.cache[:,j], L.B, X[:,j])
    end
    # Apply A to each row
    for i = 1:size(Y,1)
        @views A_mul_B!(Y[i,:], L.A, L.cache[i,:])
    end
    return y
end

# Representation
as_array(L::DiffEqKronProd) = kron(as_array(L.A), as_array(L.B))

as_array (generic function with 7 methods)

The Kronecker sum $A \oplus B$ of two square matrices is defined as $A \otimes I_m + I_n \otimes B$, where $n$ and $m$ are respectively the width of $A$ and $B$ (the name "Kronecker sum" is a bit misleading because it is not really a commutative operation; also $\oplus$ is sometimes used as the direct sum). It is frequently used to construct the Laplacian, which in $N$-dimension is

$$ \Delta = \underbrace{D^2 \oplus D^2 \oplus \cdots \oplus D^2}_{N} $$

where $D^2$ is the one-dimensional Laplacian.

Again we define the Kronecker sum using the lazy Kronecker product and the identity operators. This is not optimal though; we can probably construct a separate `DiffEqKronSum` type that works like `DiffEqOperatorCombination`.

In [11]:
function kron_sum(A::DiffEqOperator{T}, B::DiffEqOperator{T}) where {T}
    @assert size(A,1) == size(A,2) && size(B,1) == size(B,2) "Dimension mismatch!"
    Ia = DiffEqIdOperator{T}(size(A,1))
    Ib = DiffEqIdOperator{T}(size(B,1))
    kron(A,Ib) + kron(Ia,B)
end

kron_sum (generic function with 1 method)

# `L(u,p,t)` and `L(du,u,p,t)` Interface

In [12]:
# Defining functors for abstract types is currently not supported in Julia
# (L::DiffEqOperator)(u,p,t) = (update_coefficients!(L,u,p,t); L * u)
# (L::DiffEqOperator)(du,u,p,t) = (update_coefficients!(L,u,p,t); A_mul_B!(du,L,u))

# Instead, define for each subtype of DiffEqOperator
for T in subtypes(DiffEqOperator)
    (L::T)(u,p,t) = (update_coefficients!(L,u,p,t); L * u)
    (L::T)(du,u,p,t) = (update_coefficients!(L,u,p,t); A_mul_B!(du,L,u))
end

# Example

In [13]:
N = 5
a = 2.0
laplacian = spdiagm((ones(N-1), -2*ones(N), ones(N-1)), (-1,0,1))
Lap = DiffEqArrayOperator(laplacian)
A = a * DiffEqIdOperator{Float64}(N)
L = Lap + A;

In [14]:
# Internals of L
@show typeof(L)
@show L.ops
@show L.coeffs;

typeof(L) = DiffEqOperatorCombination{Float64,DefaultUpdateFunc}
L.ops = (DiffEqArrayOperator{Float64,SparseMatrixCSC{Float64,Int64},DefaultUpdateFunc}(
  [1, 1]  =  -2.0
  [2, 1]  =  1.0
  [1, 2]  =  1.0
  [2, 2]  =  -2.0
  [3, 2]  =  1.0
  [2, 3]  =  1.0
  [3, 3]  =  -2.0
  [4, 3]  =  1.0
  [3, 4]  =  1.0
  [4, 4]  =  -2.0
  [5, 4]  =  1.0
  [4, 5]  =  1.0
  [5, 5]  =  -2.0, DefaultUpdateFunc()), DiffEqIdOperator{Float64}(5))
L.coeffs = [1.0, 2.0]


In [15]:
u = rand(5)
p = nothing; t = 0.0
laplacian * u + a * u ≈ L(u,p,t)

true

In [16]:
du = zeros(5)
L(du,u,p,t)
laplacian * u + a * u ≈ du

true

In [17]:
as_array(L) ≈ laplacian + a*I

true

## Operator concatenation

In [18]:
B = DiffEqArrayOperator(rand(10,10))
L = blkdiag(Lap,B)
u = rand(15)
L * u ≈ vcat(laplacian * u[1:5], B.A * u[6:15])

true

In [19]:
du = zeros(15)
A_mul_B!(du, L, u)
du ≈ vcat(laplacian * u[1:5], B.A * u[6:15])

true

In [20]:
as_array(L)

15×15 Array{Float64,2}:
 -2.0   1.0   0.0   0.0   0.0  0.0       …  0.0         0.0       0.0      
  1.0  -2.0   1.0   0.0   0.0  0.0          0.0         0.0       0.0      
  0.0   1.0  -2.0   1.0   0.0  0.0          0.0         0.0       0.0      
  0.0   0.0   1.0  -2.0   1.0  0.0          0.0         0.0       0.0      
  0.0   0.0   0.0   1.0  -2.0  0.0          0.0         0.0       0.0      
  0.0   0.0   0.0   0.0   0.0  0.321427  …  0.302346    0.312952  0.741237 
  0.0   0.0   0.0   0.0   0.0  0.33676      0.167809    0.913174  0.215875 
  0.0   0.0   0.0   0.0   0.0  0.847332     0.00465073  0.715865  0.797013 
  0.0   0.0   0.0   0.0   0.0  0.120287     0.0160063   0.197975  0.552528 
  0.0   0.0   0.0   0.0   0.0  0.157538     0.367312    0.772254  0.933306 
  0.0   0.0   0.0   0.0   0.0  0.200154  …  0.775968    0.582819  0.222858 
  0.0   0.0   0.0   0.0   0.0  0.605551     0.846206    0.181095  0.0560462
  0.0   0.0   0.0   0.0   0.0  0.91548      0.443588    0.191295

## Multidimensional operators

Kronecker product

In [21]:
const ⊗ = kron

L1 = DiffEqArrayOperator(rand(4,5))
L2 = DiffEqArrayOperator(rand(6,7))
u = rand(5*7)
du = zeros(4*6)
L = L1 ⊗ L2;

In [23]:
L * u ≈ (L1.A ⊗ L2.A) * u

true

In [24]:
A_mul_B!(du,L,u)
du ≈ (L1.A ⊗ L2.A) * u

true

Kronecker sum

In [26]:
const ⊕ = kron_sum

Nx = 10; Ny = 20
Dxx = DiffEqArrayOperator(spdiagm((ones(Nx-1), -2*ones(Nx), ones(Nx-1)), (-1,0,1)))
Dyy = DiffEqArrayOperator(spdiagm((ones(Ny-1), -2*ones(Ny), ones(Ny-1)), (-1,0,1)))
Lap = Dxx ⊕ Dyy;

In [27]:
U = rand(Ny, Nx)
u = vec(U)
v = Lap * u
V = reshape(v, Ny, Nx)
V ≈ Dyy.A * U + U * Dxx.A

true